I use this file to create the databases. Simply put your files in a directory, update the variables below and click up to the persist. Test it out after that.

In [ ]:
where_the_dir_is = r'D:\Downloaded\BibleBuddy\server\db\Bibles\ASV'
where_the_dbs_will_be_saved = r'D:\Downloaded\BibleBuddy\server\db\Bibles\Collections\ASV'

Create DB

In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ChatVectorDBChain
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredFileLoader
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
# loader_cls = lambda path: UnstructuredFileLoader(path)
# This will allow whole directorys to be embedded. 
loader = DirectoryLoader(where_the_dir_is)
# loader_cls = lambda path: UnstructuredFileLoader(path, mode="elements", max_tokens=2000)

# loader.loader_cls = lambda path: UnstructuredFileLoader(path, mode="elements")
docs = loader.load()
len(docs)

1

In [3]:
documents = loader.load()

In [4]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
persist_directory = where_the_dbs_will_be_saved
vectorstore = Chroma.from_documents(documents, embeddings,persist_directory=persist_directory)

Running Chroma using direct local API.
No existing DB found in D:\Downloaded\BibleBuddy\server\db\Bibles\Collections\ASV, skipping load
No existing DB found in D:\Downloaded\BibleBuddy\server\db\Bibles\Collections\ASV, skipping load


In [5]:
vectorstore.persist()
vectorstore = None

Persisting DB to disk, putting it in the save folder D:\Downloaded\BibleBuddy\server\db\Bibles\Collections\ASV
PersistentDuckDB del, about to run persist
Persisting DB to disk, putting it in the save folder D:\Downloaded\BibleBuddy\server\db\Bibles\Collections\ASV


If that ^ fails, try closing some applications and try again. It took 3 tries to embed ASV Bible

Test DB

In [10]:
# Now we can load the persisted database from disk, and use it as normal. 
persist_directory = r'D:\Downloaded\BibleBuddy\server\db\Bibles\Collections\ASV'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

Running Chroma using direct local API.
loaded in 5756 embeddings
loaded in 1 collections
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [12]:
system_template="""Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. Provide a scripture reference.
ALWAYS return a "sources" part in your answer.
The "sources" part should be a reference to the source of the document from which you got your answer.

The output should be a markdown code snippet formatted in the following schema:
```json
{{
answer: is foo
sources: xyz
}}
```
Begin!
----------------
{context}"""
messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]
prompt = ChatPromptTemplate.from_messages(messages)

In [14]:
qa = ChatVectorDBChain.from_llm(ChatOpenAI(temperature=0), vectordb,qa_prompt=prompt, return_source_documents=True)


In [15]:
chat_history = []
query = "What is Exodus about?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

'Exodus is a book in the Bible that tells the story of how God delivered the Israelites from slavery in Egypt and made a covenant with them at Mount Sinai. It includes the ten plagues, the parting of the Red Sea, the giving of the Ten Commandments, and the construction of the tabernacle. \n\nSources: Exodus 1-40'

In [16]:
result['source_documents']

[Document(page_content='Exod.32:11 And Moses besought Jehovah his God, and said, Jehovah, why doth thy wrath wax hot against thy people, that thou hast brought forth out of the land of Egypt with great power and with a mighty hand?\n\nExod.32:12 Wherefore should the Egyptians speak, saying, For evil did he bring them forth, to slay them in the mountains, and to consume them from the face of the earth? Turn from thy fierce wrath, and repent of this evil against thy people.\n\nExod.32:13 Remember Abraham, Isaac, and Israel, thy servants, to whom thou swarest by thine own self, and saidst unto them, I will multiply your seed as the stars of heaven, and all this land that I have spoken of will I give unto your seed, and they shall inherit it for ever.\n\nExod.32:14 And Jehovah repented of the evil which he said he would do unto his people.', lookup_str='', metadata={'source': 'D:\\Downloaded\\BibleBuddy\\server\\db\\Bibles\\ASV\\ASV.txt'}, lookup_index=0),
 Document(page_content='Exod.12:4